### Set Context

In [ ]:
use role accountadmin;
use schema quickstart.gaming_data;

show tables;


### Basic SQL functions

In [ ]:
select * from hourly_aggregation limit 10;

In [ ]:
-- MAX BY 

select max_by(game_id, unique_users) from hourly_aggregation;

--select max_by(game_id || ' : ' || hour_event, unique_users) from hourly_aggregation;

In [ ]:
select * from events limit 10;

In [ ]:
--- GROUPING --
select game_id, count(distinct session_id) as sessoions, count(distinct user_id) as cnt_users
from events
group by all;

In [ ]:
-- EXCLUDE
select * exclude ip_address rename record_id as id
from events
limit 20;

In [ ]:
-- REGULAR JOIN

SELECT 
    l.*, 
    r.event_time, 
    DATEDIFF(minute, l.login_time, r.event_time) AS minutes_idle, 
    r.event_type
FROM dim_logins l
LEFT JOIN events r
    ON l.user_id = r.user_id 
    AND l.session_id = r.session_id
    AND l.login_time <= r.event_time
    WHERE 1=1
QUALIFY ROW_NUMBER() OVER (PARTITION BY l.login_time, l.user_id, l.session_id ORDER BY r.event_time ASC) = 1
ORDER BY l.login_time;


In [ ]:
-- AS OF JOIN

SELECT l.*, r.event_time, datediff(minute,l.login_time,r.event_time) as minutes_idle, r.event_type
  FROM dim_logins l 
  ASOF JOIN events r
    MATCH_CONDITION(l.login_time <= r.event_time)
    ON(l.user_id=r.user_id and l.session_id=r.session_id)
    --where l.user_id = 'user_3942' 
  ORDER BY l.login_time;


### Reference SQL & Python cells

In [ ]:
select * from dim_logins limit 20;

In [ ]:
events_df = cell17.to_df()

### Viz with Python

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
df = session.table('dim_games')
st.dataframe(df)

In [ ]:
df = session.sql("select game_id, count(1) as counter from raw_events group by all").to_pandas()
st.bar_chart(df,x='GAME_ID', y='COUNTER')

In [ ]:
df = session.sql("select game_id, count(1) as counter from raw_events group by all").to_pandas()
game = st.multiselect("Game",df['GAME_ID'].unique(), default=df['GAME_ID'].unique() )

filtered_df = df[df['GAME_ID'].isin(game)]
st.bar_chart(filtered_df ,x='GAME_ID' , y='COUNTER')